In [3]:
import numpy as np
import pygame
import math

np.random.seed(42)

# Constants
Nx, Ny = 50, 50
dx=min(500/Nx, 500/Ny)*2/pow(3,0.25)
du, dv = 0.03, 0.08
au, av, bu, bv = 0.08, 0.15, -0.07, 0.0
cu, cv, Du, Dv = 0.03, -0.015, 0.02, 1.2
Fmax, Gmax = 0.2, 0.5
dt = 0.1

# Initialize grid
u = np.random.rand(Nx * Ny)
v = np.random.rand(Nx * Ny)

# Neighbor index calculation with periodic boundary conditions
def get_neighbors(Nx, Ny):
    x_menos = np.zeros(Nx * Ny, dtype=int)
    x_mas = np.zeros(Nx * Ny, dtype=int)
    y_menos = np.zeros(Nx * Ny, dtype=int)
    y_mas = np.zeros(Nx * Ny, dtype=int)

    for i in range(Nx):
        for j in range(Ny):
            n = i + j * Nx
            x_menos[n] = ((i - 1) % Nx) + j * Nx
            x_mas[n] = ((i + 1) % Nx) + j * Nx
            y_menos[n] = i + 1 + ((j - 1) % Ny) * Nx
            y_mas[n] = i - 1 + ((j + 1) % Ny) * Nx
            
    return x_menos, x_mas, y_menos, y_mas

x_menos, x_mas, y_menos, y_mas = get_neighbors(Nx, Ny)
print(y_mas)

# Evolution step
def evolucion(u, v):
    aux_u = u.copy()
    aux_v = v.copy()

    for n in range(Nx * Ny):
        
        F=max(-Fmax, min((au-du) * u[n] + bu * v[n] + cu, Fmax))
        G=max(-Gmax, min(av * u[n] + (bv-dv) * v[n] + cv , Gmax))
        
        
        if(n%2 == 0):
            y_vecino = y_menos[n]
        else:
            y_vecino = y_mas[n]
        
        aux_u[n] += dt * (F + Du * (u[x_menos[n]] + u[x_mas[n]] + u[y_vecino] - 3 * u[n]))
        aux_v[n] += dt * (G + Dv * (v[x_menos[n]] + v[x_mas[n]] + v[y_vecino] - 3 * v[n]))

    u[:] = np.clip(aux_u, 0, 1)
    v[:] = np.clip(aux_v, 0, 1)

# Pygame visualization function
def visualize(u, t):
    screen.fill((0, 0, 0))

    # Get min/max values for normalization
    minimo_u = np.min(u)
    maximo_u = np.max(u)

    # Loop to create a triangular grid
    for i in range(Nx):
        for j in range(Ny):
            
            n = i + j * Nx
            if maximo_u == minimo_u:
                u_norm = u[n]
            else:
                u_norm = (u[n] - minimo_u) / (maximo_u - minimo_u)
                
            color = (int(u_norm * 255), int(u_norm * 255), int(u_norm * 255))  # Grayscale
            
            if (i%2 == 0):
                
                x = (i + j + 1) * dx/2 + 5
                y = (j + 1) * (math.sqrt(3) / 2) * dx + 5
                bottom = (x, y)
                left = (x - dx/2, y - (math.sqrt(3) / 2) * dx)
                right = (x + dx/2, y - (math.sqrt(3) / 2) * dx)
                pygame.draw.polygon(screen, color, [left, right, bottom])

            
            if (i%2 != 0):
                
                x = (i/2 + j/2 + 1/2) * dx + 5
                y = j * (math.sqrt(3) / 2) * dx + 5
                top = (x, y)
                left = (x - dx/2, y + (math.sqrt(3) / 2) * dx)
                right = (x + dx/2, y + (math.sqrt(3) / 2) * dx)
                pygame.draw.polygon(screen, color, [top, left, right])
                

    # Render time text
    time_text = font.render(f"Time: {t * dt:.2f}", True, (255, 255, 255))
    screen.blit(time_text, (10, Ny * np.sqrt(3)/2 * dx))

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((Nx * dx + 10, Ny * dx + 10))  # Extra space for time display
pygame.display.set_caption("Simulation Visualization")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 24)  # Default font, size 24
# Main loop
running = True
t = 0
visualize(u, t)
while running and t < 5000000:
    t += 1
    evolucion(u, v)

    if t % 10 == 0:  # Visualize every 10 steps
        visualize(u, t)
        #print(t)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)  # Limit to 60 FPS
        
pygame.image.save(screen, "pattern_triangular.png", dpi=600, bbox_inches='tight')
print("Saved final state as pattern_triangular.png")

pygame.quit()

[49 50 51 ... 46 47 48]
Saved final state as pattern_triangular.png


In [17]:
from PIL import Image

def save_image(u, filename="patternTriangular.png"):
    # Reshape u into a 2D array
    u_2d = u.reshape((Nx, Ny))

    # Normalize values to 0-255 for grayscale image
    u_norm = (255 * (u_2d - np.min(u_2d)) / (np.max(u_2d) - np.min(u_2d))).astype(np.uint8)

    # Save as an image
    img = Image.fromarray(u_norm, mode="L")
    img.save(filename)
    
save_image(u)

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import math

# Grid size — must match data
Nx, Ny = 50, 50
filename = "tri_result_grande.txt"
dx = 2.0 / pow(3, 0.25)  # Matching your Pygame scaling

# Load data
u = np.loadtxt(filename)
assert len(u) == Nx * Ny, "Loaded data size doesn't match Nx * Ny"

# Normalize data
min_u = np.min(u)
max_u = np.max(u)
u_norm = (u - min_u) / (max_u - min_u + 1e-8)

# Build triangle patches
patches = []
colors = []

for i in range(Nx):
    for j in range(Ny):
        idx = i + j * Nx
        val = u_norm[idx]
        gray = (val, val, val)

        if i % 2 == 0:
            x = (i + j + 1) * dx / 2
            y = (j + 1) * (math.sqrt(3) / 2) * dx
            bottom = (x, y)
            left = (x - dx / 2, y - (math.sqrt(3) / 2) * dx)
            right = (x + dx / 2, y - (math.sqrt(3) / 2) * dx)
            triangle = [left, right, bottom]
        else:
            x = (i / 2 + j / 2 + 0.5) * dx
            y = j * (math.sqrt(3) / 2) * dx
            top = (x, y)
            left = (x - dx / 2, y + (math.sqrt(3) / 2) * dx)
            right = (x + dx / 2, y + (math.sqrt(3) / 2) * dx)
            triangle = [top, left, right]

        patches.append(Polygon(triangle, closed=True))
        colors.append(gray)

# Plot with PatchCollection
fig, ax = plt.subplots(figsize=(12, 12), dpi=300)
collection = PatchCollection(patches, facecolors=colors, edgecolors=colors, linewidths=0)
ax.add_collection(collection)

ax.set_aspect('equal')
ax.autoscale_view()
ax.axis('off')

plt.tight_layout()
plt.savefig("triangular_grid_visualization.png", dpi=600, bbox_inches='tight')
plt.close()

print("✅ Image saved as triangular_grid_visualization.png")


✅ Image saved as triangular_grid_visualization.png
